<h1>Example: Convolutional Neural Network using TensorFlow (1.15) </h1>

<h2> Using tf.layers/estimator API </h2>

<h3>Convolution Neural Network</h3>

![CNN](https://raw.githubusercontent.com/worklifesg/Deep-Learning-Specialization/master/images/CNN.png)

<h3>MNIST Dataset</h3>

The MNIST database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. 

![MNIST](https://raw.githubusercontent.com/worklifesg/Deep-Learning-Specialization/master/images/MnistExamples.png)

<h4>Step 1: Import Libraries</h4>

In [1]:
import warnings
warnings.simplefilter('ignore')

In [2]:
from __future__ import division, print_function,absolute_import

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np



<h4> Step2: MNIST dataset </h4>

In [3]:
#MNSIT data using tutorials only valid in TensorFlow V1

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


<h4> Step3: Parameters </h4>

In [4]:
#Training parameters

learning_rate=0.001
num_steps=2000
batch_size=128

In [5]:
#Network parameters (MNIST data parameters)

num_input=784 # (28 x 28)
num_classes=10 # MNSIST labels or classes
dropout=0.75 # Probability keep units

<h4> Step4: Neural Network, Model function, Estimator Function </h4>

In neural network, layers is used for defining the convolution methods and functions whereas in version 2.3.0 we have direct functions and avoid using .layers and .contrib (flatten)

In [6]:
# Neural Network

def conv_total(x_dict,n_classes,dropout, reuse,is_training):
    
    with tf.variable_scope('ConvNet',reuse=reuse): #defining a scope fo reusing the variables
        x=x_dict['images']
        x=tf.reshape(x,shape=[-1,28,28,1]) #Reshape to match picture format
    
        conv1=tf.layers.conv2d(x,32,5,activation=tf.nn.relu) #1st Convolution Layer [32 filters, kernal =5]
        conv1=tf.layers.max_pooling2d(conv1,2,2) # down-sampling with kernal size 2 and strides of 2
    
        conv2=tf.layers.conv2d(conv1,64,3,activation=tf.nn.relu) #2nd Convolution Layer [32 filters, kernal =5]
        conv2=tf.layers.max_pooling2d(conv2,2,2) # down-sampling with kernal size 2 and strides of 2
    
    
        fc1=tf.contrib.layers.flatten(conv2) # Flatten data to 1D vector for fully connected layer
        fc1=tf.layers.dense(fc1,1024) #Fully connected layer with 1024 neurons
    
    
        fc1=tf.layers.dropout(fc1,rate=dropout,training=is_training) #Dropout only for true training else not applied
        
        out=tf.layers.dense(fc1,n_classes)#output layer (classes/labels)
                   
    return out


In [7]:
def model_fn(features,labels,mode):
    
    #dropout has two states leading to different behaviour at training and prediction time (2 computational graphs)
    # weights are shared here
    l_train=conv_total(features,num_classes,dropout,reuse=False,is_training=True)
    l_test=conv_total(features,num_classes,dropout,reuse=True,is_training=False)
    
    pred_classes=tf.argmax(l_test,axis=1)
    pred_probas=tf.nn.softmax(l_test) # Predictions
    
    #for early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode,predictions=pred_classes) # TF Estimators requires to return a EstimatorSpec
    
    
    #Loss and optimizer
    
    loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=l_train, labels=tf.argmax(tf.cast(labels, dtype=tf.int32),1)))
    
    #Previouslyused, will give error
    #loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
     #   logits=l_train,labels=tf.cast(labels,dtype=tf.int32)))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op=optimizer.minimize(loss,global_step=tf.train.get_global_step())
    
    # Accuracy
    
    acc= tf.metrics.accuracy(labels=tf.argmax(tf.cast(labels, dtype=tf.int32),1),
                                 predictions=pred_classes)
    
    #Previouslyused, will give error
    #acc = tf.metrics.accuracy(labels=labels,predictions=pred_classes)
    
    # TF Estimator to return the function
    
    estim_specs=tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_classes,
        loss=loss,
        train_op=train_op,
        eval_metric_ops={'accuracy': acc}    
    )
    
    return estim_specs  

<h4> Step5: Build estimator, Train Model, Evaluate Model </h4>

In [8]:
# Build estimator
warnings.filterwarnings('ignore')
model=tf.estimator.Estimator(model_fn=model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Home\\AppData\\Local\\Temp\\tmpdf3t73r8', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000242FB352EC8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
# Train model

input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)

model.train(input_fn, steps=num_steps)


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use keras.layers.dropout instead.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Home\AppData\Local\Temp\tmpdf3t73r8\model.ckpt.
INFO:tensorflow:loss = 2.3241758, step = 1
INFO:tensorflow:global_step/sec: 15.5794
INFO:tensorflow:loss = 0.17408781, step = 101 (6.434 sec)
INFO:tensorflow:global_step/sec: 16.4922
INFO:tensorflow:loss = 0.044300314, step = 201 (6.048 sec)
INFO:tensorflow:global_step/sec: 16.5088
INFO:tensorflow:loss = 0.1025812, step = 301 (6.057 sec)
INFO:tensorflow:global_step/sec: 16.3957
INFO:tensorflow:loss = 0.10141355, step = 401 (6.099 sec)
INFO:tensorflow:global_step/sec: 16.0593
INFO:tensorflow:loss = 0.04364411, st

In [10]:
# Evaluate Model

input_fn1 = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)

model.evaluate(input_fn1)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-15T23:40:19Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Home\AppData\Local\Temp\tmpdf3t73r8\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-09-15-23:40:20
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.9915, global_step = 2000, loss = 0.03535012
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2000: C:\Users\Home\AppData\Local\Temp\tmpdf3t73r8\model.ckpt-2000


{'accuracy': 0.9915, 'loss': 0.03535012, 'global_step': 2000}